In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
plt.style.use('ggplot')

Carguemos los datos en un dataframe:

In [ ]:
tabla=pd.read_csv('ETL.csv')

# 1:
Busquemos la diferencia de tiempos entre semanas:

In [ ]:
fechas=pd.DataFrame()
fechas['fecha']=pd.Series(pd.to_datetime(tabla['fecha'].value_counts().index)).sort_values()
fechas['diferencias']=fechas['fecha'].diff().astype(str).apply(lambda x: x.split()[0])
fechas.reset_index(drop=True,inplace=True)
fechas.drop(index=[0],inplace=True)
fechas['diferencias']=fechas['diferencias'].astype(int)
fechas.plot.line(x='fecha',y='diferencias',figsize=(12,6))
plt.plot(fechas['fecha'],np.ones(fechas.shape[0])*7)
plt.legend(['Diferencias en dias real','Diferencias en dias ideal(7 dias)'])
plt.show()

In [ ]:
tabla['fecha'] = pd.to_datetime(tabla['fecha'])
tabla.query("fecha>'2020-08-30' and fecha<'2020-11-01'").shape

825 registros en este periodo de tiempo equivalen a 25 pollos registrados a lo largo de dos meses.

# 2:

Ahora miremos la distribucion de edades de los pollos:

In [ ]:
(tabla['edadEnDias'].value_counts()/33).sort_index().plot.bar(figsize=(10,7))
plt.title('Cantidad de pollos por edad')
plt.ylabel('Cantidad')
plt.xlabel('Edad')
plt.show()

De aca se puede saber que a los pollos los registran mas o menos cuando su edad es un multiple de 7. ¿Esta distribucion cumple con lo esperado?

# 3:

Determinemos cuales son las 10 lesiones que mas presentan los pollos:

In [ ]:
rangos=dict(tabla.iloc[:33,[0,2]].values)
datos=tabla.query("lesionPromedio!=0").groupby('lesionTipo')['lesionPromedio'].value_counts()
orden=datos.sum(level='lesionTipo').sort_values(ascending=False).index
for i,lesion in enumerate(orden[:10]):
    print(f"#{i+1}: - {lesion} / Rango:{rangos[lesion]}")
    resultados=datos.loc[lesion].reset_index(name=lesion)
    resultados['porcentaje']=(100*resultados[lesion]/resultados[lesion].sum()).round(2)
    print(tabulate(resultados.rename(columns={lesion:'conteo'}), headers='keys', tablefmt='psql'))
    resultados.plot.bar(x='lesionPromedio',y=lesion,figsize=(8,6),fontsize=10)
    plt.title(lesion,size=15)
    plt.xticks(rotation=0)
    plt.xlabel('lesionPromedio',size=12,weight=800)
    plt.ylabel("Conteo",size=12,weight=800)
    plt.legend().remove()
    plt.grid(color='gray')
    plt.show()
    print("-----------------------------------------------------------\n\n")

Hay varias enfermedades que estan presentes en una cuarta parte de los pollos ¿Esto es de cuidado o es normal?

# 4:

Visualicemos por ultimo las 4 variables que se plantearon como de interes:

In [ ]:
def mostrar_atributo(atributo):
    if atributo=='sexoAnimales':
        resultado=tabla[atributo].replace(
        {
            'M ':'M',
            'Macho':'M',
            'Mac':'M',
            'Hem':'H',
            'Hembra':'H',
        },inplace=True)
        
    resultado=(tabla[atributo].value_counts()/33).reset_index()\
                                .rename(columns={'index':atributo,atributo:'conteo'})
    resultado['porcentaje']=(100*resultado['conteo']/resultado['conteo'].sum()).round(2)
    print(tabulate(resultado, headers='keys', tablefmt='psql'))

    resultado.plot.bar(x=atributo,y='conteo',figsize=(8,6))
    plt.xticks(rotation=0)
    plt.xlabel(atributo,size=12,weight=800)
    plt.ylabel("Conteo",size=12,weight=800)
    plt.legend().remove()
    plt.show()

In [ ]:
mostrar_atributo('sexoAnimales')
print()
mostrar_atributo('bursometro')
print()
mostrar_atributo('condicionHigado')
print()
mostrar_atributo('integridadIntestinal')